# **Pinecone tutorial**
This is the sample notebook from the Pinecone site.

How to run in Colab:
This notebook can be run on Google Colab and stand alone python development environments. Click here to run on colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datariders/tutorials/blob/main/vectordb/pinecone/pinecone_tutorial.ipynb)


References:
https://docs.pinecone.io/guides/get-started/quickstart


In [1]:
!pip3 install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 2.7 MB/s eta 0:00:00
